In [1]:
!pip install selenium


In [1]:
#Import the necessary packages

from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
service=Service(ChromeDriverManager().install())        #Initialize the selenium webdriver
options=webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])  #This will allow us to list errors if any
options.add_extension('AdBlock — block ads across the web 6.1.1.0.crx') #In order to make webscraping faster we add a adblock


In [5]:
#Loading the web driver and the website
driver = webdriver.Chrome(options=options,service=service)
driver.get('https://www.rxlist.com/drugs/alpha_n.htm')


In [6]:
#Loading beautiful soup

soup = BeautifulSoup(driver.page_source, 'html.parser')


In [7]:

soup = BeautifulSoup(driver.page_source, 'html.parser')
k=soup.find('div',class_='AZ_results')
li=k.findAll('ul')
m=0
list_sizes=[]
for i in range(len(li)):
    s=set()
    li1=li[i]
    print(i)
    print()
    li_list=li1.findChildren()
    for i in li_list:
        s.add(i.text)
    list_sizes.append(len(s))

    

0

1

2

3

4



In [9]:
#Finding all subheadings for iteration
ul=k.find('h4').find_all('a')
ulA=len(ul)

In [10]:
print(ulA)


5


In [11]:
print(list_sizes[0])

73


In [12]:

medicines_dataF=[]
m=0
wait = WebDriverWait(driver, 30)
for i in range(0,ulA):
    s=list_sizes[i]
    print(s)
    for j in range(1,s+1):
        second = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="AZ_container"]/div/ul[{i+1}]/li[{j}]/a'))) #This will make sure that page is loaded before clicking
        driver.execute_script("arguments[0].click()", second)
        soup=BeautifulSoup(driver.page_source,'html.parser') #Beautiful spup is loaded for  each webpage
        medicine_name=soup.find('h1').get_text().strip() #The h1 heading is extracted as Medicine name
        mai=soup.find('div',class_='monograph_cont')
        symptom=mai.find('p').text #The first paragraph is extracted as the use of the medicine
        dosage=mai.find_all('h4')[2].findNextSibling().text if len(mai.find_all('h4')) > 2 else "N/A" #The dosage is extracted from the 3rd h4 tag
        side=mai.findAll('li')[0:6] #The first 6 side effects are extracted
        side_effects=[]
        for effect in side:
            side_effects.append(effect.text)
        medicine_data = {
    'Medicine Name': medicine_name,
    'Use': symptom,
    'Side Effects': side_effects,
    'Dosage': dosage
}
        print(medicine_data)
        medicines_dataF.append(medicine_data)
        driver.back()
 



73


{'Medicine Name': 'Acetylcysteine Solution', 'Use': 'Mucomyst (Acetylcysteine Solution n-acetyl-l-cysteine) is a solution of a naturally occurring amino acid that is used to help clear mucus (acts as a mucolytic agent) and material entrapped in mucus in people that have thick mucus that may interfere with breathing or other functions. For example, \nMucomyst (Acetylcysteine Solution)  may be used to help clear mucus in COPD, cystic fibrosis, pneumonia, and in tracheostomy care. \nMucomyst (Acetylcysteine Solution) may also be used as an antidote for acetaminophen toxicity. \nMucomyst (Acetylcysteine Solution) is available as a generic.', 'Side Effects': ['inflammation of the mouth, ', 'nausea, ', 'vomiting, ', 'fever, ', 'runny nose, ', 'drowsiness, '], 'Dosage': 'When nebulized into a face mask, mouthpiece, or tracheostomy, 1 to 10 mL of the 20% \nMucomyst (Acetylcysteine Solution) or 2 to 20 mL of the 10% solution may be given every 2 to 6 hours; the recommended dose of \nMucomyst (A

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=125.0.6422.112)
Stacktrace:
	GetHandleVerifier [0x0025B793+45827]
	(No symbol) [0x001EDB74]
	(No symbol) [0x000E150F]
	(No symbol) [0x000CC238]
	(No symbol) [0x000CC159]
	(No symbol) [0x000E3740]
	(No symbol) [0x0015B906]
	(No symbol) [0x00142B96]
	(No symbol) [0x00116998]
	(No symbol) [0x0011751D]
	GetHandleVerifier [0x005143C3+2899763]
	GetHandleVerifier [0x005677ED+3240797]
	GetHandleVerifier [0x002E1264+593364]
	GetHandleVerifier [0x002E818C+621820]
	(No symbol) [0x001F6F54]
	(No symbol) [0x001F3658]
	(No symbol) [0x001F37F7]
	(No symbol) [0x001E58AE]
	BaseThreadInitThunk [0x759B5D49+25]
	RtlInitializeExceptionChain [0x7737B97B+107]
	RtlGetAppContainerNamedObjectPath [0x7737B901+561]
	(No symbol) [0x00000000]


In [13]:
len(medicines_dataF)

121

In [77]:
dfrm=pd.read_csv('a-b-c-d-e-f-g-h-i-j-k-l.csv')
data=pd.DataFrame(medicines_dataF)
data1=pd.concat([dfrm,data]).reset_index(drop=True)
data1

,Medicine Name,Use,Side Effects,Dosage
0,A-Methapred,A-Methapred (methylprednisolone sodium succina...,"['muscle weakness', 'peptic ulcer', 'slow woun...",Dosage recommendations for A-Methapred vary fo...
1,Kivexa,Kivexa (abacavir and lamivudine) is a combinat...,"['hives,', 'difficulty breathing,', 'swelling ...",The recommended dose of Kivexa tablets in adul...
2,Ziagen,Ziagen (abacavir sulfate) is an antiviral medi...,"['trouble sleeping, ', 'loss of appetite, ', '...",The recommended oral dose of Ziagen for adults...
3,Epzicom,Epzicom (abacavir sulfate and lamivudine) is a...,"['headache, ', 'nausea, ', 'diarrhea, ', 'dizz...",The recommended oral dose of Epzicom for adult...
4,Trizivir,"Trizivir (abacavir sulfate, lamivudine, and zi...","['headache, ', 'tiredness, ', 'weakness, ', 'n...",The recommended oral dose of Trizivir is one t...
...,...,...,...,...
903,Meruvax,Meruvax II (rubella virus) Vaccine Live is a l...,[],The dose of Meruvax II for any age is 0.5 mL a...
904,Canasa,Canasa (mesalamine) is an anti-inflammatory dr...,"[rectal pain, , pain when inserting the suppos...",The usual dosage of Canasa 1000 mg suppositori...
905,Pentasa,Pentasa (mesalamine) is an anti-inflammatory a...,"[severe stomach pain,, stomach cramping,, bloo...",The recommended dosage of Pentasa for the indu...
906,Lialda,Lialda (mesalamine) is an anti-inflammatory ag...,"[nausea, , vomiting, , stomach cramps, , diarr...",The recommended dosage of Lialda for the induc...


In [1]:
data1.to_csv('a-b-c-d-e-f-g-h-i-j-k-l-m.csv',index=True)
    

NameError: name 'data1' is not defined

In [2]:
dfrm=pd.read_csv('a-b-c-d-e-f-g-h-i-j-k-l-m.csv')

NameError: name 'pd' is not defined